In [4]:
import implicit
import numpy as np
from scipy.sparse import csr_matrix
import pandas as pd

In [5]:
data = {
    'movie_id': [0, 1, 2, 3, 4, 5, 6],
    'title': ['The Matrix', 'Blade Runner', 'Inception', 'Titanic', 'The Notebook', 'Star Wars', 'Alien']
}
df_movies = pd.DataFrame(data)

title_to_idx = {title: i for i, title in enumerate(df_movies['title'])}
idx_to_title = {i: title for title, i in title_to_idx.items()}

interactions = [
    (0, title_to_idx['The Matrix']), (0, title_to_idx['Blade Runner']), (0, title_to_idx['Star Wars']),
    (1, title_to_idx['Titanic']), (1, title_to_idx['The Notebook'])
]

In [6]:
rows = [u for u, i in interactions]
cols = [i for u, i in interactions]
data = np.ones(len(rows))
user_item_matrix = csr_matrix((data, (rows, cols)), shape=(2, len(df_movies)))

In [7]:
als_model = implicit.als.AlternatingLeastSquares(factors=32, regularization=0.01, iterations=50)
als_model.fit(user_item_matrix)

  0%|          | 0/50 [00:00<?, ?it/s]

In [8]:
new_user_likes = ['The Matrix', 'Inception']
new_user_indices = [title_to_idx[title] for title in new_user_likes]

new_user_sparse_row = csr_matrix(
    ([1] * len(new_user_indices),                       # Data (value of interaction is 1)
     ([0] * len(new_user_indices), new_user_indices)),  # Coordinates (row 0, specified columns)
    shape=(1, len(df_movies))                           # Shape of the matrix
)

In [9]:
recommended_item_ids, scores = als_model.recommend(
    userid=0,
    user_items=new_user_sparse_row,
    N=5,                                # Number of recommendations
    filter_already_liked_items=True     # Don't recommend 'The Matrix' or 'Inception' back to me
)

print(f"Because you liked: {', '.join(new_user_likes)}")
print("We recommend:")
for item_idx, score in zip(recommended_item_ids, scores):
    print(f"- {idx_to_title[item_idx]} (Score: {score:.3f})")

Because you liked: The Matrix, Inception
We recommend:
- Blade Runner (Score: 0.997)
- Star Wars (Score: 0.997)
- The Notebook (Score: 0.000)
- Titanic (Score: 0.000)
- Alien (Score: 0.000)
